In [1]:
# # Del all directories in qualitative results which are empty
# import os
# directory = "qualitative_results"
# for d in os.listdir(directory):
#     try:
#         os.rmdir(os.path.join(directory, d))
#     except:
#         print('ok')

In [ ]:
!nvidia-smi

In [3]:
! pip install tensorflow-gpu==2.3.0 # for multi-GPU training

     |████████████████████████████████| 320.4 MB 43 kB/s s eta 0:00:01
     |████████████████████████████████| 459 kB 74.9 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 27.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 78.2 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 4.9 MB 78.3 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 60.4 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.3
    Uninstalling scipy-1.5.3:
      Successfully uninstalled

In [5]:
# import tensorflow as tf
# import numpy as np

# real = np.array([[0, 0, 1, 0], [0, 1, 1, 100]])
# y=tf.math.equal(real, 0)
# print(y)
# z=tf.math.logical_not(y)
# print(z)

In [9]:
# !rm -rf qualitative_results
# !rm -rf summary
# !rm -rf models
# !rm -rf logs

In [ ]:
# !python3 model.py
!python3 main.py
# !python3 generate_recommendations.py
# !python3 dataset.py
# !python3 config.py

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
2021-07-25 08:31:09.202915: E tensorflow/core/grappler/optimizers/scoped_allocator_optimizer.cc:1138] ScopedAllocatorOptimizer: Internal: Complete shape not known for Adam/allreduce/CollectiveReduce_68
2021-07-25 08:31:09.206608: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:581] scoped_allocator_optimizer failed: Internal: Complete shape not known for Adam/allreduce/CollectiveReduce_68


In [24]:
import tensorflow as tf
import numpy as np
y_true = np.array([[1,1], [2,0]])
y_pred = np.array([[[0.05, 0.95, 0], [0.05, 0.65, 0.3]], [[0.1, 0.8, 0.1], [0.1, 0.75, 0.15]]])
# Using 'auto'/'sum_over_batch_size' reduction type.
scce = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
loss = scce(y_true, y_pred).numpy()
print(loss)

print(y_true.shape)
print(y_pred.shape)
print(loss.shape)

[[0.0512934  0.43078291]
 [2.30258512 2.30258512]]
(2, 2)
(2, 2, 3)
(2, 2)


In [ ]:
import numpy as np

# 

# EXP

In [2]:
import pandas as pd
import numpy as np

from model import rnn_reco_model
from dataset import wynk_sessions_dataset
from config import *

# Make dataset
dataset = wynk_sessions_dataset()

# Initialize model (and load the weights)
model = rnn_reco_model(dataset.vocab_size)   

LOAD_MODEL_PATH = "models/models_withtimebuckets_rnn_7_day_data_2021_06_29_200418_1024_64_10/model_00_124999"
if LOAD_MODEL:
#     model.build(input_shape=((None, MAX_LEN), (None, MAX_LEN)))    
#     print(model.summary())
    print("loading")
    model.load_weights(LOAD_MODEL_PATH)
    print(f"Model loaded: {LOAD_MODEL_PATH}")    

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7'), communication = CollectiveCommunication.AUTO

>>>>> Building vocab... >>>>>

Loading build_vocab_dict...
<<<<< Vocab built... <<<<<


>>>>> Mapping song2info... >>>>>

Loading song2info dict...
<<<<< Mapped song2info... <<<<<

loading
Model loaded: models/models_withtimebuckets_rnn_7_day_data_2021_06_29_200418_1024_64_10/model_00_124999


In [4]:
song_inp = tf.constant(np.random.randint(1, 3, (1,10)))
time_bucket_inp = tf.constant(np.random.randint(1, 3, (1,10)))
print("song_inp.shape: ", song_inp.shape)
print("time_bucket_inp.shape: ", time_bucket_inp.shape)

# q()
_ = model(song_emb_inp=song_inp,
          time_bucket_emb_inp=time_bucket_inp)

song_inp.shape:  (1, 10)
time_bucket_inp.shape:  (1, 10)


In [8]:
# for i in model.layers:
#     print(type(i), i.name, type(i.weights), len(i.weights))

# q()
# x = model.song_emb.weights[0]
x = model.time_bucket_emb.weights[0]

print(type(x), x.shape)

np.sum(x.numpy())
# x


#         print(model.summary())
#         model.load_weights(LOAD_MODEL_PATH)


<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> (229, 32)


39.466763

In [12]:
'''
# new model
song 83.67507
time -1.1676371

# trainied model
song -199908.17
time 38.30059
'''

'\n# new model\nsong 83.67507\ntime -1.1676371\n\n# trainied model\nsong -199908.17\ntime 38.30059\n'

In [30]:
# test_song_ids_df = pd.read_csv(QUALITATIVE_TEST_DATA_PATH)
# print(test_song_ids_df.head(20))

# test_song_ids = ["srch_unisysinfo_M09050388", "srch_sonymusic_INS171602370", "srch_sonymusic_698050", "srch_hungama_396873"]
# for s in test_song_ids:
#     print(s, dataset.song2info[s])

# srch_unisysinfo_M09050388 TITLE: Dil Diyan Gallan | ALBUM: Tiger Zinda Hai | ARTIST: ,  | FREQUENCY: 217086 | LANG: hindi | YEAR: 2017
# srch_sonymusic_INS171602370 TITLE: Channa Mereya | ALBUM: Ae Dil Hai Mushkil | ARTIST:  | FREQUENCY: 309043 | LANG: hindi | YEAR: 2016
# srch_sonymusic_698050 TITLE: Maa Tujhe Salam | ALBUM: Vande Mataram Maa Tujhe Salaam | ARTIST:  | FREQUENCY: 21259 | LANG: hindi | YEAR: 2006
# srch_hungama_396873 TITLE: Kun Faya Kun | ALBUM: Rockstar | ARTIST: Mohit Chauhan,A R Rahman,Javed Ali | FREQUENCY: 168921 | LANG: hindi | YEAR: 2011

In [4]:
song_inp = dataset.item2idx["srch_hungama_396873"] # Kun Faya Kun
song_inp = dataset.item2idx["srch_unisysinfo_M09050388"] # Dil Diyan Gallan
song_inp = dataset.item2idx["srch_sonymusic_INS171602370"] # Channa Mereya
song_inp = dataset.item2idx["srch_sonymusic_698050"] # Maa Tujhe Salam

def get_next_song_pred(song_inp, time_bucket_inp, num_recommendation_timesteps=1):
    song_inp = np.array([[song_inp]])
    time_bucket_inp = np.array([[time_bucket_inp]])
    
    exclude = np.zeros([1, dataset.vocab_size]) #(1, vocab_size)

    for idx in song_inp[0]:
        exclude[0, idx] = 1 

    initial_state = None #[state_h, state_c]
    
    for t in range(num_recommendation_timesteps):
        probs, state_h, state_c = model(song_emb_inp=song_inp,
                                        time_bucket_emb_inp=time_bucket_inp,
                                        training=False, 
                                        initial_state=initial_state) # probs (1, num_items)    
        
        probs = probs*(1-exclude)             # probs (1, vocab_size)
        pred = tf.math.argmax(probs, axis=-1) # (1, )
        
        exclude[0, int(pred[0])] = 1 #(1, vocab_size)

        inp = tf.expand_dims(pred, axis = -1) # (1, 1)        
        initial_state = [state_h, state_c]
        next_song_pred = int(pred[0])
                        
    print(f"time {time_bucket_inp[0, 0]}: {int(pred[0])}: {probs[0, int(pred[0])].numpy()}")
    return next_song_pred

for t in [int(i) for i in np.linspace(0, 228, num=50)]:
    print("================")
    next_song_pred = get_next_song_pred(song_inp=song_inp, time_bucket_inp=t)
    print(dataset.song2info[dataset.idx2item[next_song_pred]])



time 0: 17997: 0.01167581882327795
TITLE: Bharat Humko Jaan Se Pyara Hai | ALBUM: Roja | ARTIST: Hariharan,A R Rahman | FREQUENCY: 2980 | LANG: hindi | YEAR: 1994
time 4: 5611: 0.00964123010635376
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 9: 5611: 0.00972586777061224
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 13: 5611: 0.01098481472581625
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 18: 5611: 0.010092789307236671
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 23: 5611: 0.009190144948661327
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,S